In [1]:
import pandas as pd
import altair as alt

# Seminar: the Long-Run Prices Dataset

A notebook using the LRPD, described [here](https://cep.lse.ac.uk/pubs/download/occasional/op055.pdf).

</br></br></br></br>


Let's take a look at the prices dataset first

In [2]:
prices_df = pd.read_parquet('https://autocpi-public.s3.eu-west-2.amazonaws.com/lrpd/db_prices.parquet')
prices_df.describe().round()

,quote_date,shop_code,item_id_raw,region,price,item_id
count,48368958.0,48368958.0,48368973.0,48368958.0,48368958.0,48368973.0
mean,200776.0,477.0,388041.0,7.0,50.0,388398.0
std,1060.0,1532.0,146756.0,3.0,206.0,146672.0
min,198802.0,1.0,210101.0,1.0,0.0,210101.0
25%,199811.0,39.0,212917.0,3.0,1.0,212918.0
50%,200805.0,88.0,430128.0,7.0,5.0,430132.0
75%,201707.0,802.0,510406.0,9.0,20.0,510407.0
max,202510.0,20071.0,640406.0,13.0,44000.0,640406.0


We've got 48 million observations and a dates from 1982 to 2025.
</br></br></br></br>


We also need the items data to understand what each product is.

In [3]:
items_df = pd.read_parquet('https://autocpi-public.s3.eu-west-2.amazonaws.com/lrpd/db_item.parquet')
items_df.head()

,item_id,description,date_quote_s,date_quote_e,n_obs
0,210101,LARGE LOAF-WHITE-SLICED-800G,198802,200401,36039
1,210102,LARGE LOAF-WHITE-UNSLICED-800G,198802,202510,56917
2,210105,LARGE WHOLEMEAL LOAF-UNSLICED,198802,200301,27161
3,210106,SIX BREAD ROLLS-WHITE/BROWN,198802,202510,67469
4,210107,"BROWN LOAF,400G,SLICED-GRAN",198903,200401,29361


</br></br></br>
# Simple chart: the price of milk

Let's chart the price of milk. First we need to work out what the `item_id` is. Let's look in the items_df to find out.
</br></br>

In [4]:
items_df[items_df ['date_quote_e'] ==202510]

,item_id,description,date_quote_s,date_quote_e,n_obs
1,210102,LARGE LOAF-WHITE-UNSLICED-800G,198802,202510,56917
3,210106,SIX BREAD ROLLS-WHITE/BROWN,198802,202510,67469
8,210111,WHITE SLICED LOAF BRANDED 750G,200402,202510,36848
10,210113,WHOLEMEAL SLICED LOAF BRANDED,200402,202510,35696
11,210114,CHILLED GARLIC BREAD,201002,202510,41672
...,...,...,...,...,...
1380,640226,PRIVATE HEALTH CLUB ANNUAL FEE,200102,202510,34014
1383,640240,LIVERY CHARGES PER WEEK,200802,202510,21546
1384,640243,SOFT PLAY SESSION TIME PERIOD,201802,202510,11098
1385,640244,CLIMBING WALL SESSION,202202,202510,2149


In [9]:
items_df['description'].str.contains("bread", case=False)

,description
0,False
1,False
2,False
3,True
4,False
...,...
1382,False
1383,False
1384,False
1385,False


In [11]:
items_df[items_df['description'].str.contains('onions', case=False)]

,item_id,description,date_quote_s,date_quote_e,n_obs
256,212507,FRESH VEG-ONIONS-PER LB,198802,199812,39910
266,212519,FRESH VEG-ONIONS-PER KG,199902,202510,102804


In [12]:
items_df[items_df['description'].str.contains('milk', case=False)]

,item_id,description,date_quote_s,date_quote_e,n_obs
155,211701,SHOP MILK-PASTEURISED-PER PT,198802,200201,32488
156,211709,SHOP MILK-WHOLE MILK-4PT/2LTR,199002,202510,91066
157,211710,MILK SEMI-PER 2 PINTS/1.136 L,199002,202510,65794
158,211711,MILK-FLAVOURED,199602,200301,11750
159,211712,"MILK, LONG-LIFE, 500ML",200002,200201,3013
160,211713,"MILK FLAVOURED, 180-500MLS/GMS",201402,202510,28711
161,211714,NON-DAIRY MILK DRINK 900ML-1LT,201702,202510,24039
164,211804,POWDERED SKIMMED MILK-198G,198903,200201,24594
171,211812,MILK- FLAVOURED 500-568ML,200302,200601,4253
373,213004,HOT MILK DRINK-300-400G,198802,200601,34269


Let's go for `211710` - `MILK SEMI-PER 2 PINTS/1.136 L`. It's got a long timeseries (1992-202510) and is probably representative.

</br></br></br></br>

First, let's just plot the mean, median and median price of milk over time.

In [14]:
prices_df['item_id'] == 211711

,item_id
0,False
1,False
2,False
3,False
4,False
...,...
48368968,False
48368969,False
48368970,False
48368971,False


In [15]:
milk_prices = prices_df.query("item_id == 211711")
avg_milk_prices = milk_prices.groupby('quote_date').agg({'price': ['mean', 'median']}).reset_index()

avg_milk_prices.columns = ['date', 'Mean', 'Median']
avg_milk_prices

,date,Mean,Median
0,199602.0,0.466786,0.420
1,199603.0,0.507647,0.430
2,199604.0,0.506845,0.430
3,199605.0,0.508588,0.430
4,199606.0,0.515345,0.435
...,...,...,...
79,200209.0,0.700511,0.790
80,200210.0,0.708062,0.790
81,200211.0,0.697154,0.790
82,200212.0,0.694615,0.650


In [16]:
milk_prices

,quote_date,shop_code,item_id_raw,region,price,indicator_box,item_id
5125691,199704.0,247.0,211711,9.0,0.35,,211711
5125692,200107.0,802.0,211711,11.0,0.61,,211711
5125693,200212.0,941.0,211711,3.0,0.85,C,211711
5125694,199707.0,802.0,211711,9.0,0.77,,211711
5125695,199701.0,911.0,211711,4.0,0.75,,211711
...,...,...,...,...,...,...,...
5137436,200212.0,103.0,211711,3.0,0.95,,211711
5137437,200003.0,40.0,211711,9.0,0.35,,211711
5137438,200002.0,71.0,211711,4.0,0.43,,211711
5137439,200108.0,807.0,211711,3.0,0.59,,211711


To use it in Altair/Vega-lite, we just have to melt it from wide to long.

In [18]:
milk_prices = prices_df[prices_df['item_id'] ==211711].copy()
milk_prices

,quote_date,shop_code,item_id_raw,region,price,indicator_box,item_id
5125691,199704.0,247.0,211711,9.0,0.35,,211711
5125692,200107.0,802.0,211711,11.0,0.61,,211711
5125693,200212.0,941.0,211711,3.0,0.85,C,211711
5125694,199707.0,802.0,211711,9.0,0.77,,211711
5125695,199701.0,911.0,211711,4.0,0.75,,211711
...,...,...,...,...,...,...,...
5137436,200212.0,103.0,211711,3.0,0.95,,211711
5137437,200003.0,40.0,211711,9.0,0.35,,211711
5137438,200002.0,71.0,211711,4.0,0.43,,211711
5137439,200108.0,807.0,211711,3.0,0.59,,211711


In [19]:
avg_milk_prices_melted = avg_milk_prices.melt(id_vars=['date'],
                                                   var_name='price_type',
                                                   value_name='price')

# and we can also just format the date nicely, so altair treats them properly
avg_milk_prices_melted['date'] = pd.to_datetime(avg_milk_prices_melted['date'], format='%Y%m')

avg_milk_prices_melted

,date,price_type,price
0,1996-02-01,Mean,0.466786
1,1996-03-01,Mean,0.507647
2,1996-04-01,Mean,0.506845
3,1996-05-01,Mean,0.508588
4,1996-06-01,Mean,0.515345
...,...,...,...
163,2002-09-01,Median,0.790000
164,2002-10-01,Median,0.790000
165,2002-11-01,Median,0.790000
166,2002-12-01,Median,0.650000


In [20]:
alt.Chart(avg_milk_prices_melted).mark_line().encode(
    x=alt.X('date:T', title=''),
    y=alt.Y('price:Q', title='Price (GBP)'),
    color='price_type:N'
).properties(
    title={
        "text": "Milk Prices",
        "subtitle": ["Mean and Median Prices for Milk-flavoured (2 Pints)", "Source: ONS microdata via Davies (2021)"],
        "fontSize": 16
    }
)

alt.Chart(...)